In [1]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
#from apache_beam.utils.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflo
w Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)

In [8]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
import json


with beam.Pipeline(InteractiveRunner(),options=options) as p:
    stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/json-timestamp.json')
    stage2 = stage1| "Make Dictionary">>beam.FlatMap(lambda x: json.loads(x)) #Use FlatMap instead of Map to unpack the list of dictionary items
    stage3 = stage2| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-timestamped-table',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='Name:STRING,Score:INTEGER,Time:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    
ib.show_graph(p)    
ib.show(stage1)
ib.show(stage2)

/usr/bin/dot


In [9]:
# Dataflow Runner

import apache_beam as beam
from apache_beam.runners import DataflowRunner


p = beam.Pipeline(options=options) #option must be given here and when dataflowrunner is called, otherwise an error will occur. To resolve that error, some mechanism also exists.

stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/json-timestamp.json')
stage2 = stage1| "Make Dictionary">>beam.FlatMap(lambda x: json.loads(x)) #Use FlatMap instead of Map to unpack the list of dictionary items
stage3 = stage2| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-timestamped-table',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='Name:STRING,Score:INTEGER,Time:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )

pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip
